In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('/Users/namchangseok/Desktop/코드스테이츠/AI_12기/cp2/2019-Oct.csv')

In [3]:
#시간을 없애고 날짜만 남김
df['event_time']=pd.to_datetime(df['event_time']).dt.date

In [4]:
df= df.drop(['user_session','event_time','category_id','category_code','brand'], axis=1)

In [5]:
df.to_csv('price_id_data.csv')

In [3]:
df=pd.read_csv('/Users/namchangseok/Desktop/코드스테이츠/AI_12기/cp2/price_id_data.csv')
df.head(1)

,Unnamed: 0,event_type,product_id,price,user_id
0,0,view,44600062,35.79,541312140


In [9]:
df=df.sample(n=3000)

In [5]:
from sklearn.model_selection import train_test_split
train,valid = train_test_split(df, test_size=0.2,random_state=42)

In [6]:
import numpy as np
# mAP
def apk(actual, predicted, k=20):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [9]:
valid_df = valid.groupby('user_id', as_index=False).agg({'product_id': list})
train_df = train.groupby('user_id', as_index=False).agg({'product_id': list})
train_df

,user_id,product_id
0,33869381,[7002639]
1,64078358,[10600284]
2,183503497,[22200103]
3,184265397,"[27400002, 27400002, 6902133, 6902133, 6902303]"
4,200673532,"[6501011, 6701171, 6701171]"
...,...,...
2867193,566280663,[1005127]
2867194,566280676,[13201002]
2867195,566280697,[2300307]
2867196,566280780,[15100003]


In [10]:
popular_items = train.drop_duplicates(['product_id', 'user_id']).groupby('product_id', as_index=False).size().sort_values('size', ascending=False)
popular_items


,product_id,size
919,1004856,175259
843,1004767,155266
1121,1005115,148589
1111,1005105,99132
898,1004833,87049
...,...,...
105601,25502405,1
105600,25502403,1
105599,25502402,1
41117,10701143,1


In [11]:
valid_df = valid_df.merge(train_df, on='user_id')
valid_df

,user_id,product_id_x,product_id_y
0,184265397,[6902303],"[27400002, 27400002, 6902133, 6902133, 6902303]"
1,200673532,[6501011],"[6501011, 6701171, 6701171]"
2,205053188,[17301541],[17301541]
3,209714031,"[8700258, 8700583, 26900028, 44500029, 5500007]","[5500268, 26900028, 5500007, 26900028, 4450002..."
4,210507313,[7101160],[7101227]
...,...,...,...
1712627,566279704,"[13102963, 13100770, 13102412]","[13100770, 13100770, 13100770, 5701331, 13102963]"
1712628,566280169,[11500195],"[11500195, 11500195]"
1712629,566280399,[4804695],"[4802888, 4804110, 4804110, 4803399, 4803186]"
1712630,566280422,"[5700967, 5701038]",[5701038]


In [16]:
valid_df['pred'] = [[i for i in list(popular_items['product_id'][:10].values) if i not in j][:10] for j in valid_df['product_id_y']]
valid_df.head(10)

,user_id,product_id_x,product_id_y,pred
0,184265397,[6902303],"[27400002, 27400002, 6902133, 6902133, 6902303]","[1004856, 1004767, 1005115, 1005105, 1004833, ..."
1,200673532,[6501011],"[6501011, 6701171, 6701171]","[1004856, 1004767, 1005115, 1005105, 1004833, ..."
2,205053188,[17301541],[17301541],"[1004856, 1004767, 1005115, 1005105, 1004833, ..."
3,209714031,"[8700258, 8700583, 26900028, 44500029, 5500007]","[5500268, 26900028, 5500007, 26900028, 4450002...","[1004856, 1004767, 1005115, 1005105, 1004833, ..."
4,210507313,[7101160],[7101227],"[1004856, 1004767, 1005115, 1005105, 1004833, ..."
5,219406386,[13300001],"[13200475, 13200176, 13200918, 13200221]","[1004856, 1004767, 1005115, 1005105, 1004833, ..."
6,228761834,[7202310],[7202310],"[1004856, 1004767, 1005115, 1005105, 1004833, ..."
7,237271696,[5100818],"[5100818, 5100816, 5100816]","[1004856, 1004767, 1005115, 1005105, 1004833, ..."
8,239475080,"[3100198, 3100198]",[11300053],"[1004856, 1004767, 1005115, 1005105, 1004833, ..."
9,239876607,"[1005033, 1004957, 1004956, 1005014]","[1004856, 1004956, 1005102, 1004956, 1005099, ...","[1004767, 1005115, 1005105, 1004833, 1004249, ..."


In [15]:
mapk(valid_df['product_id_x'], valid_df['pred'], 10)

0.011712640271363758

In [21]:
def create_ranked_df(train):
        product_ratings = train.groupby('product_id', as_index=False).size()['size']
        num_ratings = train.drop_duplicates(['product_id', 'user_id']).groupby('product_id', as_index=False).size()['size']
        last_rating = train.groupby('product_id', as_index=False).agg({'event_time': max}).rename(columns={'event_time': 'last_rating'})
        
        rating_count_df = pd.DataFrame({'avg_rating': product_ratings, 'num_ratings': num_ratings})
        rating_count_df = rating_count_df.join(last_rating)

        recs =  train.loc[:, ['product_id', 'category_id', 'category_code', 'brand', 'price']].drop_duplicates().merge(rating_count_df, on=['product_id'])

        ranked = recs.sort_values(['avg_rating', 'num_ratings', 'last_rating'], ascending=False)

        ranked = ranked[ranked['num_ratings'] > 4]
        
        return ranked
    

def popular_recommendations(user_id, n_top, ranked):

    top = list(ranked['product_id'][:n_top])

    return top

In [22]:
#베이스라인 모델 테스트 
test=df.loc[df['user_id']==	523013344]
test=test.groupby('user_id', as_index=False).agg({'product_id': list})

In [22]:
test

,user_id,product_id
0,523013344,"[1700718, 1700718, 28717396, 1801631, 28717467]"
